In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd
from urllib.parse import quote
from tqdm.notebook import tqdm
import folium, json

In [2]:
# 관서명 주소 전화번호
# csv 파일로 저장
# kakao api로 위도 경도
# 못받아온 경우 인덱스 확인후 drop
# 페이지를 바꾸는 것은 셀레늄으로 바꾸고 난후는 BS로 크롤링

In [3]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword=&pageIndex=1'     
#pageIndex=2

In [4]:
headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
}

In [5]:
url_i = f'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword=&pageIndex={i}' 

NameError: name 'i' is not defined

In [15]:
driver = webdriver.Chrome()
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword=&pageIndex=1'     
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
soup

<html class="sub" lang="ko"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
<title>전국소방기관 안내&lt; 일반현황&lt; 기관소개&lt; 소방청</title>
<meta content="소방청" name="subject"/>
<meta content="소방청" name="description"/>
<meta content="소방청,알림·소식,정보공개,민원,제안,정책자료,기관소개" name="Keywords"/>
<!-- <link rel="shortcut icon" href="favicon.ico" type="image/x-icon" /> -->
<link href="/nfa/common/css/font-awesome.min.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/nfa/common/css/jquery.mCustomScrollbar.min.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/nfa/common/css/style.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/nfa/common/css/sub.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/nfa/common/css/board.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/nfa/c

In [8]:
name = soup.find('span', attrs = {'class', 'title'}).get_text()
name

'마포소방서-서교-119 안전센터'

In [9]:
addr = soup.find('address').get_text()
addr

'서울특별시 마포구 양화로 60 (서교동)'

In [10]:
tel = soup.find('span', attrs={'class', 'tel'}).get_text()
tel

'02-3143-2119'

In [16]:
lines = []
for page in tqdm(range(1, 9)):
    url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword=&pageIndex=' + str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    uls = soup.find('ul', attrs = {'class', 'stations-list'}).find_all('li')
    # odd, even number of pages should have different click
    for ul in uls:
        name = ul.find('span', attrs = {'class', 'title'}).get_text()
        addr = ul.find('address').get_text()
        tel = ul.find('span', attrs={'class', 'tel'}).get_text()
        lines.append([name, addr, tel])

  0%|          | 0/8 [00:00<?, ?it/s]

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=112.0.5615.121)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0071DCE3+50899]
	(No symbol) [0x006AE111]
	(No symbol) [0x005B5588]
	(No symbol) [0x0059D333]
	(No symbol) [0x005FF4DB]
	(No symbol) [0x0060DB33]
	(No symbol) [0x005FB6F6]
	(No symbol) [0x005D7708]
	(No symbol) [0x005D886D]
	GetHandleVerifier [0x00983EAE+2566302]
	GetHandleVerifier [0x009B92B1+2784417]
	GetHandleVerifier [0x009B327C+2759788]
	GetHandleVerifier [0x007B5740+672048]
	(No symbol) [0x006B8872]
	(No symbol) [0x006B41C8]
	(No symbol) [0x006B42AB]
	(No symbol) [0x006A71B7]
	BaseThreadInitThunk [0x77450099+25]
	RtlGetAppContainerNamedObjectPath [0x77E77B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77E77B3E+238]


In [ ]:
uls = soup.find('ul', attrs = {'class', 'stations-list'}).find_all('li')
uls

In [ ]:
len(uls)

In [ ]:
uls[2]

In [ ]:
lines

In [ ]:
df = pd.DataFrame(lines, columns=['name', 'address', 'telephone'])
df
# for column in df.columns[3:]:
#     df[column] = df[column].apply(convert_unit)

In [ ]:
df.to_csv('소방서주소.csv', index=False)

In [20]:
df = pd.read_csv('소방서주소.csv')
df.head()

,name,address,telephone
0,마포소방서-서교-119 안전센터,서울특별시 마포구 양화로 60 (서교동),02-3143-2119
1,마포소방서,서울특별시 마포구 창전로 76(신수동),02-716-0604
2,마포소방서-염리-119 안전센터,서울특별시 마포구 토정로 37길 27 (염리동),02-713-0965
3,마포소방서-신수-119 안전센터,서울특별시 마포구 창전로 76 (신수동),02-716--9121
4,영등포소방서-당산-119 안전센터,서울특별시 영등포구 양평로 70-1 (당산동6가),02-2633-0119


In [21]:
with open('./kakaoAPI.txt') as file:
    kakao_key = file.read()

In [22]:
lat_list = []
lng_list = []
for i in df.index:
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df.address[i])}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
    lat_list.append(lat)
    lng_list.append(lng)

IndexError: list index out of range

In [ ]:
df.index

In [32]:
search_url = "https://dapi.kakao.com/v2/local/search/address.json"
url = f'{search_url}?query={quote(df.address[i])}'
result = requests.get(url, headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%9D%80%ED%8F%89%EA%B5%AC%20%EC%88%98%EC%83%89%EB%A1%9C%20294%20%28%EC%88%98%EC%83%89%EB%8F%99%29'

In [24]:
result

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [33]:
df.address[i]
#서울특별시 종로구 종로1길 36(수송동)

'서울특별시 은평구 수색로 294 (수색동)'

In [26]:
result['documents']

[]

In [27]:
print(df.address[i])

서울특별시 은평구 수색로 294 (수색동)


str